In [7]:
from ibl_pipeline.analyses import behavior as behavior_analyses
from ibl_pipeline import behavior as behavior
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
import plotting_utils as putils
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

In [31]:
subj_uuid = (subject.Subject & 'subject_nickname="IBL_13"').fetch1('subject_uuid')
key = {'subject_uuid': subj_uuid,
   'session_start_time': datetime.datetime(2019, 3, 22, 13, 1, 28)}
sessions = behavior_analyses.PsychResultsBlock * behavior_analyses.ReactionTimeContrastBlock & key

data = []

for session in sessions.fetch('KEY'):
    contrasts, prob_left, reaction_time, ci_low, ci_high = \
        (sessions & session).fetch1(
            'signed_contrasts', 'prob_left', 'reaction_time_contrast', 
            'reaction_time_ci_low', 'reaction_time_ci_high')
    error_low = reaction_time - ci_low
    error_high = ci_high - reaction_time
    
    contrasts = contrasts * 100
    
    curve_color, error_color = putils.get_color(prob_left, 0.3)

    rt_data = go.Scatter(
        x=contrasts.tolist(),
        y=reaction_time.tolist(),
        marker=dict(
            size=6,
            color=curve_color),
        mode='markers+lines',
        name=f'p_left = {prob_left}'
    )
    
    rt_errorbar = go.Scatter(
        x=contrasts.tolist(),
        y=reaction_time.tolist(),
        error_y=dict(
            type='data',
            array=error_high.tolist(),
            arrayminus=error_low.tolist(),
            color=error_color,
            visible=True
        ),
        marker=dict(
            size=6),
        mode='none',
        name='median with 68% CI',
    )

    data.append(rt_data)
    data.append(rt_errorbar)

layout = go.Layout(
    width=630,
    height=350,
    title=dict(
        text='Reaction time - Contrast',
        x=0.25,
        y=0.85
    ),
    xaxis=dict(
        title='Contrast (%)'), 
    yaxis=dict(
        title='Probability choosing right',
        range=[-0.05, 1.05]),
    legend=dict(
        x=1.1, 
        y=0.9,
        orientation='v'),
)


fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:151: UserWarning:

Mysql server has gone away. Reconnecting to the server.



In [25]:
f = open("session_reaction_time_contrast.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()